# <center>Basic Scraping (i guess?)</center>

<center>Author : Ramos Janoah - 13514089</center>

# Before scrapping: 
#### Decide tools or library or anything to request web page and parse the html page 

In [ ]:
from urllib.request import urlopen # --> f}or request
from bs4 import BeautifulSoup # --> for scrapping

# First, decide which page you want to scraped

For example, this site: https://www.bukalapak.com/c/handphone/hp-smartphone?from=omnisearch&search_source=omnisearch_redirect

# Second, decide what detail do you want to scrape

For example, from that site, i want to scrape the name of products, and the price, for allllll the products, iterately..

<center>![Big picture, Small Picture](big_picture_small_picture.png)</center>

In the url example, the big picture is the list of products, meanwhile, the small picture if detail of product.
It depends on you, which page do you want to scrape first. Is it the 'big picture', or the 'small picture'.

For this time, small picture first. 
Step:
- See the raw html text for the aspect you want
- If you found it, make sure that it also applied to another page

In [ ]:
# for example, the page is:
url = 'https://www.bukalapak.com/p/handphone/hp-smartphone/i59i3c-jual-bb-davis-resmi-tam-like-new?dtm_section=top_promoted&dtm_source=list-product&from=&product_owner=normal_seller'
page = urlopen(url).read()

In [ ]:
str(page)[0:1000] + '....'

## Now, let BeautifoulSoup parse the HTML

In [ ]:
soup = BeautifulSoup(page, 'html.parser')

If i want to get price, get the <b>IDENTIFIER TAG OF THE ASPECT YOU WANT</b>.

For example, for the price of products

In [ ]:
price = soup.find("meta", {"name":"product:price:amount"})

or the name of products..

In [ ]:
name = soup.find("title")
name.text

Well, you can clean that later...

Now, you can make function to easily scrap for 'small picture', like this:

In [ ]:
def scrape_small_picture(url):
    result = []
    page = urlopen(url).read()
    soup = BeautifulSoup(page, 'html.parser')

    # get the name
    name = soup.find("title").text
    result.append(name)    
    
    # get the price
    price = soup.find("meta", {"name":"product:price:amount"})['content']
    result.append(price)
    
    # and get all of your sh** here....
    
    # get the bla bla
    # get the bla bla
    return result
    

### Let's check~

In [ ]:
scrape_small_picture(url)

# Tadaa, small picture is done! Easy, right?

# Now for the big picture!

In [ ]:
big_picture_url = 'https://www.bukalapak.com/c/handphone/hp-smartphone?from=omnisearch&search_source=omnisearch_redirect'
big_page = urlopen(big_picture_url).read()
big_soup = BeautifulSoup(big_page, 'html.parser')

In [ ]:
print(str(big_soup)[0:1000])

## Now, we want to get all of the 'small picture URL'

In [ ]:
r = big_soup.findAll("article", {"class":"product-display"})[0]

str(r)[0:1000] + '..'

In [ ]:
r['data-url']

In [ ]:
print('https://www.bukalapak.com' + r['data-url'])

In [ ]:
first = big_soup.findAll("article", {"class":"product-display"})[0]
str(first)[0:1000]

# Lets iterate!! (1)

In [ ]:
big_picture_url = 'https://www.bukalapak.com/c/handphone/hp-smartphone?from=omnisearch&search_source=omnisearch_redirect'

def get_url_of_the_big_picture(big_picture_url):
    # initiate list to contain url
    small_picture_url = []
    
    # open and parse big soup
    big_page = urlopen(big_picture_url).read()
    big_soup = BeautifulSoup(big_page, 'html.parser')
    
    # get div of big soup
    products_div = big_soup.findAll("article", {"class":"product-display"})
    
    # insert one by one
    for div in products_div:
        small_picture_url.append(div['data-url'])

    return small_picture_url
        
        
small_picture_url = get_url_of_the_big_picture(big_picture_url)
small_picture_url[0:5]

### Now lets iterate to get the all small picture..

In [ ]:
for i, url in enumerate(small_picture_url):
    url = 'http://www.bukalapak.com/' + url
    products_info = scrape_small_picture(url)
    print(products_info)
    
    if i == 5:
        break

# Now, how if the big picture is have the bigger picture?

# Let's iterate over the page!

### See the 'page=' in the end of string?

In [ ]:
template_url = 'https://www.bukalapak.com/c/handphone/hp-smartphone?from=omnisearch&search_source=omnisearch_redirect&page='

In [ ]:
i = 0 

# Iterately, get the big picture
while True:
    list_url = template_url + str(i)
    detail_urls = get_url_of_the_big_picture(list_url)

    # and iterately, get the small picture
    for detail_url in detail_urls:
        result = scrape_small_picture('https://www.bukalapak.com' + detail_url)
        print(result)